# Avance 1: Proyecto

## Entendimiento y preparación de los datos

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [11]:
# Ruta del archivo
file_path = "Datos_proyecto.xlsx"

# Cargar el Excel
df = pd.read_excel(file_path, sheet_name="Sheet1")

# Mostrar las primeras filas
df.head()

,textos,labels
0,"""Aprendizaje"" y ""educación"" se consideran sinó...",4
1,Para los niños más pequeños (bebés y niños peq...,4
2,"Además, la formación de especialistas en medic...",3
3,En los países de la OCDE se tiende a pasar de ...,4
4,Este grupo se centró en las personas que padec...,3


In [12]:
# Ver información general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2424 entries, 0 to 2423
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   textos  2424 non-null   object
 1   labels  2424 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 38.0+ KB


Separamos nuestros conjuntos de entrenamiento y prueba

In [13]:
# Separar train/test
X_train, X_test, y_train, y_test = train_test_split(
    df["textos"], df["labels"], test_size=0.2, random_state=42
)

Ahora empezaremos construyendo el pipeline de preprocesamiento y entrenamiento de nuestros datos. Pero antes revisaremos TfidfVectorizer que nos ayudará a preprocesar nuestras entradas y a construir nuestra bag of words (BOW).

In [14]:
import unicodedata
import nltk
from nltk.corpus import stopwords

# Se preparan las stopwords en español para utilizar en vectorizer
nltk.download('stopwords')

# Se quitan los acentos de las stopwords, lo cual también es importante (evitar warnings)
def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFKD', s)
                   if not unicodedata.combining(c))


spanish_stopwords = sorted({ strip_accents(w.lower()) for w in stopwords.words('spanish') })
len(spanish_stopwords), spanish_stopwords[:20]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(306,
 ['a',
  'al',
  'algo',
  'algunas',
  'algunos',
  'ante',
  'antes',
  'como',
  'con',
  'contra',
  'cual',
  'cuando',
  'de',
  'del',
  'desde',
  'donde',
  'durante',
  'e',
  'el',
  'ella'])

In [15]:
# Usamos solo una muestra pequeña de tus datos para verlos "a mano"
muestra = df["textos"].head(5)

# Crear el vectorizador
vectorizer = TfidfVectorizer(
            stop_words=spanish_stopwords,   # stop words en español
            lowercase=True,                 # minúsculas
            strip_accents='unicode',      # quitar los acentos: educación -> educacion
            min_df=2,                       # ignora términos que aparezcan en <2 docs
            max_df=0.9,                     # ignora términos muy frecuentes >90%
            ngram_range=(1,2),              # se incluyen bigramas
            sublinear_tf=True,
            max_features=20000,           # evita sobrecargas por bigramas
)

# Ajustar y transformar
X_tfidf = vectorizer.fit_transform(muestra)

# Ver las palabras del vocabulario que creó TF-IDF
print("Palabras en el vocabulario:\n", vectorizer.get_feature_names_out())

Palabras en el vocabulario:
 ['cada' 'cada vez' 'forma' 'formacion' 'mental' 'mentales' 'nivel' 'ocde'
 'pueden' 'salud' 'salud mental' 'servicios' 'servicios salud'
 'trastornos' 'trastornos mentales' 'tratamiento' 'vez']


## Modelado y Evaluación

Ahora construiremos los pipelines para el preprocesamiento y entrenamiento de nuestros datos de prueba

In [16]:
tfidf = TfidfVectorizer(
    stop_words=sorted(spanish_stopwords),
    lowercase=True,
    strip_accents='unicode',
    min_df=2,          # ignora términos muy raros
    max_df=0.9,        # ignora términos demasiado frecuentes
    ngram_range=(1,2), # unigrams + bigrams
    sublinear_tf=True, # tf = 1 + log(tf) AJUSTE PARA EL CONTEO, MEJOR LOGARITMICO
    max_features=20000 # límite para no explotar RAM
)


**Pipeline 1 (Regresión Logística)**

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Pipeline: TF-IDF + Logistic Regression
pipe_logreg = Pipeline([
    ("tfidf", tfidf),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",  # maneja desbalance de clases
        multi_class="auto"
    ))
])

pipe_logreg.fit(X_train, y_train)
y_pred_logreg = pipe_logreg.predict(X_test)

print("=== Regresión Logística ===")
print(classification_report(y_test, y_pred_logreg, digits=3))


c:\ProgramData\anaconda3\envs\profiling\lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


=== Regresión Logística ===
              precision    recall  f1-score   support

           1      0.956     0.964     0.960       112
           3      0.971     0.982     0.976       168
           4      0.995     0.980     0.988       205

    accuracy                          0.977       485
   macro avg      0.974     0.976     0.975       485
weighted avg      0.978     0.977     0.977       485



**Pipeline 2 (SVM Máquinas de Vectores de Soporte)**

In [18]:
from sklearn.svm import LinearSVC

# Pipeline: TF-IDF + Linear SVM
pipe_svm = Pipeline([
    ("tfidf", tfidf),
    ("clf", LinearSVC(class_weight="balanced", dual=True))
])

pipe_svm.fit(X_train, y_train)
y_pred_svm = pipe_svm.predict(X_test)

print("=== Support Vector Machine (Lineal) ===")
print(classification_report(y_test, y_pred_svm, digits=3))

=== Support Vector Machine (Lineal) ===
              precision    recall  f1-score   support

           1      0.956     0.964     0.960       112
           3      0.965     0.982     0.973       168
           4      0.995     0.976     0.985       205

    accuracy                          0.975       485
   macro avg      0.972     0.974     0.973       485
weighted avg      0.976     0.975     0.975       485



**Pipeline 3 (Bayes Ingenuo Multinomial)**

In [19]:
from sklearn.naive_bayes import MultinomialNB

# Pipeline: TF-IDF + Multinomial Naive Bayes
pipe_nb = Pipeline([
    ("tfidf", tfidf),
    ("clf", MultinomialNB(alpha=0.5))  # alpha suaviza probabilidades (HIPERPARAMETRO?)
])

pipe_nb.fit(X_train, y_train)
y_pred_nb = pipe_nb.predict(X_test)

print("=== Naive Bayes Multinomial ===")
print(classification_report(y_test, y_pred_nb, digits=3))


=== Naive Bayes Multinomial ===
              precision    recall  f1-score   support

           1      0.969     0.830     0.894       112
           3      0.943     0.976     0.959       168
           4      0.940     0.985     0.962       205

    accuracy                          0.946       485
   macro avg      0.950     0.931     0.938       485
weighted avg      0.947     0.946     0.945       485



**Optimización de Hiperparámetros – Pipeline 3 (Multinomial Naive Bayes)**

Con el fin de mejorar el rendimiento del clasificador **Multinomial Naive Bayes** aplicado sobre representaciones TF-IDF, se implementó un proceso de búsqueda exhaustiva de hiperparámetros mediante **GridSearchCV** con validación cruzada estratificada (5 folds).  

##### Parámetros evaluados
- **Vectorización TF-IDF**
  - `ngram_range`: (1,1) y (1,2) → solo unigramas vs. unigramas + bigramas  
  - `min_df`: [1, 2, 5] → frecuencia mínima para incluir términos  
  - `max_df`: [0.85, 0.95] → frecuencia máxima para descartar términos muy comunes  
  - `sublinear_tf`: [True] → transformación log(1+tf) para estabilizar pesos  
  - `strip_accents`: ['unicode'] → normalización de acentos  
  - `stop_words`: [None, spanish_stopwords] → sin filtrado vs. lista de stopwords en español

- **Selección de características**
  - `SelectKBest(chi2, k=5000/10000)` → mantuvo solo los términos más informativos  
  - `"passthrough"` → sin selección, se mantienen todas las features  

- **Clasificador MultinomialNB**
  - `alpha`: [0.1, 0.5, 1.0, 2.0] → suavizado de Laplace para evitar probabilidades cero  
  - `fit_prior`: [True, False] → usar priors ajustados a la frecuencia de clases vs. priors uniformes  

##### Métrica de evaluación
Se utilizó **F1-macro** como métrica principal, ya que promedia equitativamente el rendimiento en todas las clases, evitando sesgos hacia la clase mayoritaria.  

##### Mejores parámetros encontrados
```python
{
 'clf__alpha': 1.0,
 'clf__fit_prior': False,
 'select': SelectKBest(chi2, k=5000),
 'tfidf__max_df': 0.85,
 'tfidf__min_df': 2,
 'tfidf__ngram_range': (1, 2),
 'tfidf__stop_words': None,
 'tfidf__strip_accents': 'unicode',
 'tfidf__sublinear_tf': True
}


In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import numpy as np

pipe_nb = Pipeline(steps=[
    ("tfidf", TfidfVectorizer()),
    ("select", "passthrough"),
    ("clf", MultinomialNB())
])

# Vmos a definir un grid de hiperparámetros para buscar la mejor configuración
# Usamos SelectKBest con χ² para selección de características (o no)

param_grid_nb = {
    # TF-IDF: mejorar representación
    "tfidf__ngram_range": [(1,1), (1,2)],             # Queremos unigramas y bigramas porque hay terminos u frases claves en pares
    "tfidf__min_df": [1, 2, 5],                       # filtra términos muy raros
    "tfidf__max_df": [0.85, 0.95],                    # filtra términos muy frecuentes
    "tfidf__sublinear_tf": [True],                    # Transformar TF a log(1+tf), reduce peso de términos muy frecuentes
    "tfidf__strip_accents": ["unicode"],              # normaliza acentos
    "tfidf__stop_words": [None, spanish_stopwords], # stop words en español

    # Selección de características χ² (o no)
    "select": [SelectKBest(chi2, k=5000),
               SelectKBest(chi2, k=10000),
               "passthrough"],

    # NB: suavizado y priors
    "clf__alpha": [0.1, 0.5, 1.0, 2.0],               # suavizado de Laplace: “simula” como si todas las palabras aparecieran al menos una vez.
                                                      # Evita probabilidades cero para palabras no vistas en una clase.

    "clf__fit_prior": [True, False],                  # usar priors aprendidos vs uniformes
                                                      # En clases desbalanceadas, priors aprendidos suelen ayudar
}


# Se divide el conjunto de entrenamiento en 5 folds estratificados
# para mantener la proporción de clases en cada fold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

gs_nb = GridSearchCV(
    estimator=pipe_nb, # el pipeline base a optimizar
    param_grid=param_grid_nb, # el grid de hiperparámetros
    scoring="f1_macro", 
    cv=cv,
    n_jobs=-1,
    verbose=1
)

gs_nb.fit(X_train, y_train)

print("Mejores hiperparámetros encontrados:")
print(gs_nb.best_params_)
print(f"Mejor F1_macro (CV): {gs_nb.best_score_:.4f}")

# Evaluación en test
y_pred = gs_nb.predict(X_test)
print("\n==> TEST (MNB mejorado)")
print(classification_report(y_test, y_pred, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))



Fitting 5 folds for each of 576 candidates, totalling 2880 fits
Mejores hiperparámetros encontrados:
{'clf__alpha': 1.0, 'clf__fit_prior': False, 'select': SelectKBest(k=5000, score_func=<function chi2 at 0x000001F52F43DBD0>), 'tfidf__max_df': 0.85, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2), 'tfidf__stop_words': None, 'tfidf__strip_accents': 'unicode', 'tfidf__sublinear_tf': True}
Mejor F1_macro (CV): 0.9707

==> TEST (MNB mejorado)
              precision    recall  f1-score   support

           1     0.9464    0.9464    0.9464       112
           3     0.9760    0.9702    0.9731       168
           4     0.9757    0.9805    0.9781       205

    accuracy                         0.9691       485
   macro avg     0.9661    0.9657    0.9659       485
weighted avg     0.9691    0.9691    0.9691       485

F1_macro test: 0.9658883631892673
Matriz de confusión:
 [[106   2   4]
 [  4 163   1]
 [  2   2 201]]


In [ ]:
# Pipeline 3 – Multinomial Naive Bayes (versión con mejores parámetros)

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Definir el pipeline con los mejores hiperparámetros
nb_final = Pipeline(steps=[
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode",
        stop_words=None
    )),
    ("select", SelectKBest(score_func=chi2, k=5000)),
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# Entrenar
nb_final.fit(X_train, y_train)

# Predicciones en test
y_pred = nb_final.predict(X_test)

# Evaluación
print("==> Resultados Pipeline 3")
print(classification_report(y_test, y_pred, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))


==> Resultados Pipeline 3 (NB Final)
              precision    recall  f1-score   support

           1     0.9464    0.9464    0.9464       112
           3     0.9760    0.9702    0.9731       168
           4     0.9757    0.9805    0.9781       205

    accuracy                         0.9691       485
   macro avg     0.9661    0.9657    0.9659       485
weighted avg     0.9691    0.9691    0.9691       485

F1_macro test: 0.9658883631892673
Matriz de confusión:
 [[106   2   4]
 [  4 163   1]
 [  2   2 201]]


#### Celda: Pipeline Naive Bayes con combinación de word n-grams y char n-grams

En esta celda se implementó un **nuevo pipeline** para el clasificador Naive Bayes, 
enfocado en enriquecer la representación de los textos mediante la combinación de 
**n-gramas de palabras** y **n-gramas de caracteres**.

##### ¿Qué se hace?
1. **Vectorización doble del texto**:
   - **Word n-grams (1,2)**: genera unigramas y bigramas para capturar tanto palabras sueltas 
     como expresiones clave (*“salud pública”*, *“educación calidad”*).  
   - **Char n-grams (3–5)**: genera secuencias de 3 a 5 caracteres para capturar 
     sufijos/prefijos (*“-ción”*, *“edu-”*), variaciones morfológicas y errores de escritura.  
2. **Unión de representaciones**: ambas salidas TF-IDF se combinan con `FeatureUnion`, 
   creando un espacio de características más rico.  
3. **Selección de características**: se usa `SelectKBest(chi2, k=10000)` para quedarse con 
   las 10.000 más informativas y reducir ruido.  
4. **Clasificación**: se entrena un **Multinomial Naive Bayes** con `alpha=1.0` y 
   `fit_prior=False` (priors uniformes para balancear clases).

##### ¿Qué se busca?
- **Mejorar la robustez** frente a errores ortográficos o variaciones en las palabras.  
- **Capturar contexto semántico y morfológico** al mismo tiempo, combinando niveles 
  diferentes de análisis (palabra y carácter).  
- **Aumentar el F1-macro** y lograr un modelo más equilibrado en la clasificación de ODS.

##### ¿Por qué?
- Los **word n-grams** son muy buenos para detectar expresiones directamente relacionadas 
  con los ODS, pero pueden fallar si hay errores de escritura.  
- Los **char n-grams** complementan al modelo al detectar patrones sub-léxicos que son 
  comunes incluso con variaciones (ej. “educacion” vs “educación”).  
- La combinación aporta una señal más completa, y la selección de características 
  evita que el modelo se vea afectado por ruido excesivo.

##### Resultado esperado
Con esta celda se consiguió un rendimiento superior al pipeline anterior:  
- **F1-macro en test = 0.9735** (vs 0.9659 del mejor pipeline anterior).  
- Matriz de confusión más limpia y mejor balance entre las tres clases.  


In [ ]:
# Pipeline NB con mezcla de word n-grams (1-2) + char n-grams (3-5)
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, confusion_matrix

# Bloque de features: une dos TF-IDF sobre el MISMO texto (word + char)
features_union = FeatureUnion(transformer_list=[
    ("tfidf_word", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode"
        stop_words=None
    )),
    ("tfidf_char", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=2,
        sublinear_tf=True
        # max_df por defecto (=1.0) suele ir bien para char n-grams
    )),
])

nb_charword_final = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),   # puedes ajustar k
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# Entrenar
nb_charword_final.fit(X_train, y_train)

# Evaluar
y_pred_cw = nb_charword_final.predict(X_test)
print("==> NB word+char")
print(classification_report(y_test, y_pred_cw, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred_cw, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_cw))


==> NB word+char
              precision    recall  f1-score   support

           1     0.9550    0.9464    0.9507       112
           3     0.9880    0.9762    0.9820       168
           4     0.9808    0.9951    0.9879       205

    accuracy                         0.9773       485
   macro avg     0.9746    0.9726    0.9735       485
weighted avg     0.9773    0.9773    0.9773       485

F1_macro test: 0.9735340121177855
Matriz de confusión:
 [[106   2   4]
 [  4 164   0]
 [  1   0 204]]


In [24]:
# Pipeline 3 Final – Multinomial Naive Bayes optimizado (word + char n-grams)
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, confusion_matrix

# Vectorización combinada
features_union = FeatureUnion(transformer_list=[
    ("tfidf_word", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode",
        stop_words=None  # No usar stop words para capturar más patrones
    )),
    ("tfidf_char", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=2,
        sublinear_tf=True
    )),
])

# Pipeline final
nb_pipeline_final = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# Entrenar
nb_pipeline_final.fit(X_train, y_train)

# Predicciones en test
y_pred_final = nb_pipeline_final.predict(X_test)

# Evaluación
print("==> Resultados Pipeline 3 Final (NB Word+Char Optimizado)")
print(classification_report(y_test, y_pred_final, digits=4))
print("F1_macro test:", f1_score(y_test, y_pred_final, average="macro"))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_final))


==> Resultados Pipeline 3 Final (NB Word+Char Optimizado)
              precision    recall  f1-score   support

           1     0.9550    0.9464    0.9507       112
           3     0.9880    0.9762    0.9820       168
           4     0.9808    0.9951    0.9879       205

    accuracy                         0.9773       485
   macro avg     0.9746    0.9726    0.9735       485
weighted avg     0.9773    0.9773    0.9773       485

F1_macro test: 0.9735340121177855
Matriz de confusión:
 [[106   2   4]
 [  4 164   0]
 [  1   0 204]]


## Analisis de Resultados

In [25]:
# === Comparación final: Regresión Logística vs SVM lineal vs NB (word+char + χ²) ===
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (
    classification_report, confusion_matrix,
    accuracy_score, precision_recall_fscore_support
)
import pandas as pd

# --- Representación: word n-grams (1–2) + char n-grams (3–5) ---
features_union = FeatureUnion(transformer_list=[
    ("tfidf_word", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.85,
        sublinear_tf=True,
        strip_accents="unicode",
        stop_words=None
    )),
    ("tfidf_char", TfidfVectorizer(
        analyzer="char",
        ngram_range=(3, 5),
        min_df=2,
        sublinear_tf=True
    )),
])

# --- Pipelines de los 3 modelos ---
pipe_logreg = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),
    ("clf", LogisticRegression(
        solver="liblinear",      
        multi_class="ovr",
        max_iter=2000
        
    ))
])

pipe_svm = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),
    ("clf", LinearSVC(C=1.0))    # SVM lineal (rápida en BoW/TF-IDF)
])

pipe_nb_final = Pipeline(steps=[
    ("vec", features_union),
    ("select", SelectKBest(score_func=chi2, k=10000)),
    ("clf", MultinomialNB(alpha=1.0, fit_prior=False))
])

# --- Función de evaluación y resumen ---
def evaluar_y_resumir(nombre, clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(f"\n{'='*20} {nombre} {'='*20}")
    print(classification_report(y_test, y_pred, digits=4))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

    acc = accuracy_score(y_test, y_pred)
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(
        y_test, y_pred, average="macro", zero_division=0
    )
    p_weight, r_weight, f1_weight, _ = precision_recall_fscore_support(
        y_test, y_pred, average="weighted", zero_division=0
    )
    return {
        "modelo": nombre,
        "accuracy": acc,
        "precision_macro": p_macro,
        "recall_macro": r_macro,
        "f1_macro": f1_macro,
        "precision_weighted": p_weight,
        "recall_weighted": r_weight,
        "f1_weighted": f1_weight,
    }

# --- Correr y comparar ---
resultados = []
for nombre, modelo in [
    ("Regresión Logística", pipe_logreg),
    ("SVM lineal", pipe_svm),
    ("Naive Bayes (word+char + χ²)", pipe_nb_final),
]:
    resultados.append(evaluar_y_resumir(nombre, modelo, X_train, y_train, X_test, y_test))

df_comp = pd.DataFrame(resultados).sort_values(by="f1_macro", ascending=False).reset_index(drop=True)
print("\n==> Comparación de modelos (test):")
display(df_comp)  # en Jupyter; si no, usa print(df_comp.to_string(index=False))


c:\ProgramData\anaconda3\envs\profiling\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\profiling\lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(



==================== Regresión Logística ====================
              precision    recall  f1-score   support

           1     0.9901    0.8929    0.9390       112
           3     0.9489    0.9940    0.9709       168
           4     0.9808    0.9951    0.9879       205

    accuracy                         0.9711       485
   macro avg     0.9732    0.9607    0.9659       485
weighted avg     0.9719    0.9711    0.9707       485

Matriz de confusión:
 [[100   8   4]
 [  1 167   0]
 [  0   1 204]]

==================== SVM lineal ====================
              precision    recall  f1-score   support

           1     0.9640    0.9554    0.9596       112
           3     0.9708    0.9881    0.9794       168
           4     0.9901    0.9805    0.9853       205

    accuracy                         0.9773       485
   macro avg     0.9750    0.9746    0.9748       485
weighted avg     0.9774    0.9773    0.9773       485

Matriz de confusión:
 [[107   3   2]
 [  2 166   0]
 

,modelo,accuracy,precision_macro,recall_macro,f1_macro,precision_weighted,recall_weighted,f1_weighted
0,SVM lineal,0.977320,0.974957,0.974647,0.974762,0.977386,0.977320,0.977312
1,Naive Bayes (word+char + χ²),0.977320,0.974559,0.972580,0.973534,0.977296,0.977320,0.977269
2,Regresión Logística,0.971134,0.973244,0.960676,0.965930,0.971872,0.971134,0.970719


In [ ]:
# === Exportar comparación a Excel
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    classification_report, confusion_matrix
)

# 1) Recalcular/asegurar df_comp, reportes y matrices
modelos = [
    ("SVM lineal",                      pipe_svm),
    ("Naive Bayes (word+char + χ²)",    pipe_nb_final),
    ("Regresión Logística",             pipe_logreg),
]

filas, reportes, cms = [], {}, {}
etiquetas = sorted(set(y_test))

for nombre, clf in modelos:
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    p_m, r_m, f1_m, _ = precision_recall_fscore_support(y_test, y_pred, average="macro", zero_division=0)
    p_w, r_w, f1_w, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted", zero_division=0)
    filas.append({
        "modelo": nombre,
        "accuracy": acc,
        "precision_macro": p_m,
        "recall_macro": r_m,
        "f1_macro": f1_m,
        "precision_weighted": p_w,
        "recall_weighted": r_w,
        "f1_weighted": f1_w,
    })
    reportes[nombre] = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).T
    cms[nombre] = pd.DataFrame(
        confusion_matrix(y_test, y_pred, labels=etiquetas),
        index=[f"true_{c}" for c in etiquetas],
        columns=[f"pred_{c}" for c in etiquetas]
    )

df_comp = pd.DataFrame(filas).sort_values("f1_macro", ascending=False).reset_index(drop=True)
print("==> Comparación de modelos (test)")
print(df_comp.to_string(index=False))

# 2) Exportar a Excel
excel_path = "comparacion_modelos_LR_SVM_NB.xlsx"

def exportar_a_excel(path, resumen, reps, mats):
    try:
        # Primer intento: openpyxl (suele venir en Anaconda)
        with pd.ExcelWriter(path, engine="openpyxl") as writer:
            resumen.to_excel(writer, sheet_name="resumen", index=False)
            for nombre, rep in reps.items():
                rep.to_excel(writer, sheet_name=f"reporte_{nombre[:20]}", index=True)
            for nombre, cm in mats.items():
                cm.to_excel(writer, sheet_name=f"cm_{nombre[:20]}", index=True)
        print(f"\nArchivo Excel guardado con openpyxl: {path}")
        return True
    except ModuleNotFoundError:
        try:
            # Segundo intento: xlsxwriter
            with pd.ExcelWriter(path, engine="xlsxwriter") as writer:
                resumen.to_excel(writer, sheet_name="resumen", index=False)
                for nombre, rep in reps.items():
                    rep.to_excel(writer, sheet_name=f"reporte_{nombre[:20]}", index=True)
                for nombre, cm in mats.items():
                    cm.to_excel(writer, sheet_name=f"cm_{nombre[:20]}", index=True)
            print(f"\nArchivo Excel guardado con xlsxwriter: {path}")
            return True
        except ModuleNotFoundError:
            return False

ok = exportar_a_excel(excel_path, df_comp, reportes, cms)


==> Comparación de modelos (test)
                      modelo  accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  recall_weighted  f1_weighted
                  SVM lineal  0.977320         0.974957      0.974647  0.974762            0.977386         0.977320     0.977312
Naive Bayes (word+char + χ²)  0.977320         0.974559      0.972580  0.973534            0.977296         0.977320     0.977269
         Regresión Logística  0.971134         0.973244      0.960676  0.965930            0.971872         0.971134     0.970719

Archivo Excel guardado con openpyxl: comparacion_modelos_LR_SVM_NB.xlsx
